In [ ]:
"""
SCORE NATIVE SEQUENCES ON IMPORTED STRUCTURES 

This script is for scoring experimentally solved structures with both COREX and Rosetta.

The only input for this script are a list of PDB codes. This import is made at the beginning of the script 
directly after the import cell - this is the only cell you are required to edit.

The output file will contain all the structures scored by Rosetta (in REU) 
and COREX (in Total LogOdds, also referred to as the Fold Recognition Score)

"""


In [1]:
"""
IMPORTS
"""

import Bio
from Bio.PDB import *
from biopandas.pdb import PandasPdb
from Bio.PDB import PDBList
import csv
from datetime import date, datetime
import itertools
from itertools import chain
import math      
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np; np.random.seed(1)
import os
import pandas as pd
import subprocess
from subprocess import call, PIPE, Popen
from subprocess import PIPE, STDOUT
import seaborn as sns
import scipy.stats
from scipy.stats import pearsonr
from scipy.stats import norm
import shutil
import statistics
import time
import urllib.request

# Imports and initiates PyRosetta
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
init("-relax:constrain_relax_to_start_coords "
     "-use_input_sc "
     "-relax:ramp_constraints false "
     "-ex1 "
     "-ex2 "
     "-packing:flip_HNQ "
     "-no_optH "
     "-no_optH false")
# Init a score function from rosetta
sf = pyrosetta.get_fa_scorefxn()
# Init fast relax from rosetta protocols
fr = pyrosetta.rosetta.protocols.relax.FastRelax(sf)

# Record today's date for file generation. Beware! If generating multiple files on the
# same day, this will overwrite unless you intervene 
today = date.today()
date = today.strftime("%y%m%d")
now = datetime.now()
current_time = now.strftime("%H:%M:%S")

print("Today is", date)
print("The time is", current_time)

# Establish the current directory 
i_am_here = os.getcwd()

# Create internal directories for output files if they aren't already present
# Name paths for these directories to better organize output files 

# Find the PDB directory if it exists, otherwise create one
if not os.path.exists('%s/PDBs'%i_am_here):
    os.makedirs('%s/PDBs'%i_am_here)
pdb_dir = '%s/PDBs'%i_am_here

# Find the Rosetta directory if it exists, otherwise create one
if not os.path.exists('%s/Rosetta'%i_am_here):
    os.makedirs('%s/Rosetta'%i_am_here)
ros_dir = '%s/Rosetta'%i_am_here

# Find the COREX directory if it exists, otherwise create one
if not os.path.exists('%s/COREX_Output'%i_am_here):
    os.makedirs('%s/COREX_Output'%i_am_here)
cor_dir = '%s/COREX_Output'%i_am_here
corex_files = '%s/COREX_Files'%i_am_here

native_logodds = pd.read_csv('%s/Native_Log_Odds.csv'%corex_files)
denat_logodds = pd.read_csv('%s/Denatured_Log_Odds.csv'%corex_files)

aa_dict = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}


PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python37.mac 2020.20+release.c522e9e9054813e62f9415bcc29478af67fee549 2020-05-14T10:55:54] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python37.mac r255 2020.20+release.c522e9e9054 c522e9e9054813e62f9415bcc29478af67fee549 http://www.pyrosetta.org 2020-05-14T10:55:54
core.init: command: PyRosetta -relax:constrain_relax_to_start_coords -use_input_sc -relax:ramp_constraints false -ex1 -ex2 -packing:flip_HNQ -no_optH -no_optH false -database /Users/keilasheetz/opt/anaconda3/lib/python3.7/site-packages/pyrosetta-2020.20+release.c522e9e9054-py3.7-macosx-10.9-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1979938160 seed_offset=0 real_seed=1979938160
basic.random.init_ran

In [2]:
"""
ESTABLISH ALL IMPORT FILES
"""
# Establish the path to the files containing the PDB structures to import 
# Establish the path to the sequences to thread on to the PDB structures and a unique identifier

# CSV file of PDB codes
pdb_csv = 'your_pdbs.csv'

pdb_list = []
with open(pdb_csv, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        pdb_list.append(row)
        
pdb_list = list(chain.from_iterable(pdb_list))
print(pdb_list)


['2dyrF', '2dyrG', '4ubpA', '5fhcJ', '5hmgB']


In [7]:
"""
IMPORT THE LIST OF PDBs AND DOWNLOAD THEM 
"""
# This script will download the PDB files necessary if they are not already in the current directory
# You can skip this step if you have already downloaded the pdb files 

print(pdb_list)
errors = []

ppdb0 = []
ppdb1 = []
pdbl = PDBList()

for i in pdb_list:
    try:
        code = i[0:4]
        print(code)
        ppdb0.append(pdbl.retrieve_pdb_file(code,pdir='.', file_format ='pdb'))
        ppdb1.append(PandasPdb().fetch_pdb(code))
        
        shutil.move(i_am_here + '/pdb%s.ent'%code.lower(), pdb_dir + '/pdb%s.ent'%code.lower())
   
    except Exception as e: 
        print(e)
        print('%s was not downloaded'%i)
        errors.append(i)
        

['2dyrF', '2dyrG', '4ubpA', '5fhcJ', '5hmgB']
2dyr
2dyr
4ubp
5fhc
5hmg


In [8]:
"""
CLEAN ALL FILES
"""
# Remove any hydrogens so that COREX can handle all PDB files
# Right now this can handle NMR PDBs, but will ONLY take model 1
# If you would like to select a different model, it should be manually done before running this cell

# Iterate through the directory and find all PDBs that have not been processed
for pdb in pdb_list:
    
    for file in os.listdir(pdb_dir):

        if file.endswith(".ent") and pdb[0:4].lower() or pdb[0:4] in file:

            pdb_name = file[3:7]
            print(pdb_name)
            
            if len(pdb)==5:
                chain = pdb[4]
                print(chain)
            else:
                chain = 'A'

            if "%s.cleaned.pdb" %pdb not in os.listdir(pdb_dir):

                # Open the file and only keep non-H atoms
                # If there are multiple models, only keep model 1 atoms

                name = (os.path.join('%s/PDBs/'%i_am_here, file))
                fo = open(name, "r")
                lines = fo.readlines()

                cleanlist = []

                for line in lines:
                    if line[0:4] == "ATOM" and line[21:22]==chain:
                        if not line[77:78] == 'H':
                            if line[16:17] == ' ' or line[16:17] == 'A':
                                cleanlist.append(line)

                    elif line[0:6] == "ENDMDL":
                        break

                cleanlist.append('END')

                with open('%s.cleaned.pdb' %pdb, 'w') as filehandle:
                    for item in cleanlist:
                        filehandle.write('%s' %item)

                fo.close()

                shutil.move('%s/%s.cleaned.pdb'%(i_am_here, pdb), 
                            '%s/%s.cleaned.pdb'%(pdb_dir, pdb))


2dyr
F
2dyr
G
4ubp
A
5fhc
J
5hmg
B


In [ ]:
"""
CREATE THE NATIVE POSES WITH ROSETTA
"""
# Score all PDBs with their native structure
# The output file contains all of the PDB scores, sequences, and runtimes

scores = []

for pdb in pdb_list:
    
    start = time.time()
    
    pose = pyrosetta.pose_from_pdb("%s/%s.cleaned.pdb"%(pdb_dir, pdb))
    seq = pose.sequence()
    
    # Minimize the pose using fast relax and the full atom score function
    fr.apply(pose)
    # Score the post (ensures score data is included in the PDB file dump)
    score = sf(pose)

    pose.dump_pdb('%s/%s.rosetta.pdb'%(ros_dir, pdb))

    end = time.time()
    runtime = end - start

    scores.append([pdb, seq, score, runtime])

    df = pd.DataFrame(mutant_scores, columns = ['PDB', 'Sequence', 'Rosetta Score', 'Runtime'])
    df.to_csv('%s_rosetta_data.csv'%date)
    

In [ ]:
"""
ENSEMBLE GENERATION
"""
# Create ensemble (.5.4) files for all proteins
# If the ensemble file is already in the directory, it will skip that PDB 

# This outputs the progress, useful if multiple PDBs are being processed
counter = 0
# Find the total number of files that will be run by this cell 
for file in os.listdir(pdb_dir):
    if file.endswith(".cleaned.pdb") and "%s.5.4"%file not in os.listdir(cor_dir) \
    and "%s.MC.5.4"%file not in os.listdir(cor_dir):
        counter += 1


exceptions = []
        
run_counter = 0
for file in os.listdir(pdb_dir):
    if file.endswith(".cleaned.pdb") and "%s.5.4"%file not in os.listdir(i_am_here) \
    and "%s.MC.5.4"%file not in os.listdir(i_am_here):
        
        pdb_name = file.replace(".cleaned.pdb", "")

        # Record the name of the file in byte form to input into COREX
        bytename = bytes('%s/%s'%(pdb_dir, file), 'utf-8')

        # Anticipate file names to avoid duplicate runs
        reg_file = file + ".5.4"
        MC_file = file + ".MC.5.4"
        print(file)
        print(reg_file)

        file_name = (os.path.join(pdb_dir, file))
        fo = open(file_name, "r")
        lines = fo.readlines()
        size = len(lines)
        
        print(file[0:4], "has ", size, "atoms")

        if size < 550 and reg_file not in os.listdir(cor_dir):
            try:
                # Run EnsembleGenerator for each PDB, then run a.out for each entropy scaling factor
                subprocess.run('./COREX_Files/EnsembleGenerator', input=(bytename), shell=False, check=True)
                
                shutil.move('%s/%s.cleaned.pdb.5.4'%(pdb_dir, pdb_name), '%s/%s.cleaned.pdb.5.4'%(i_am_here, pdb_name))
                shutil.move('%s/%s.cleaned.pdb.info'%(pdb_dir, pdb_name), '%s/%s.cleaned.pdb.info'%(i_am_here, pdb_name))

                run_counter += 1
                print()
                print("You have run %s of %s files so far"%(run_counter, counter))
                print()

            
            except Exception as e: 
                print(e)
                exceptions.append(file[0:4])

        if size > 550 and MC_file not in os.listdir(cor_dir):
            try:
                # Run EnsembleGenerator for each PDB, then run a.out for each entropy scaling factor
                subprocess.run('./COREX_Files/EnsembleGeneratorMC', input=(bytename), shell=False, check=True)
                
                shutil.move('%s/%s.cleaned.pdb.MC.5.4'%(pdb_dir, pdb_name), '%s/%s.cleaned.pdb.MC.5.4'%(i_am_here, pdb_name))
                shutil.move('%s/%s.cleaned.pdb.info'%(pdb_dir, pdb_name), '%s/%s.cleaned.pdb.info'%(i_am_here, pdb_name))

                run_counter += 1
                print()
                print("You have run %s of %s files so far"%(run_counter, counter))
                print()
                

            except Exception as e: 
                print(e)
                exceptions.append(file[0:4])
        
print(exceptions)


In [ ]:
"""
CREATE THERMODESCIPT FILES
"""
# Calls the relavent a.out program to generate thermodescript files 

problemos = []

# Iterate through directory and only run this section on files not processed
for file in os.listdir(i_am_here):
    if file.endswith(".5.4"):

        check = file.replace(".5.4", "")

        if "%s.W5.T25S0.500.ThermoDescriptN"%check not in os.listdir(i_am_here):

            # Identify which are MC files and which are not
            # Generate the native and denatured ThermoDescript files

            try:
            
                if 'MC' in file:
                    name = file.replace(".MC.5.4", "")
                    byte_command = bytes("%s\n\n\n\n\n"%name, 'utf-8')

                    native_program = "./COREX_Files/a.out_nativeMC"
                    denatured_program = "./COREX_Files/a.out_denaturedMC"

                    ps = subprocess.Popen(native_program , shell=False, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
                    ps.communicate(input = byte_command)[0]

                    ps = subprocess.Popen(denatured_program , shell=False, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
                    ps.communicate(input = byte_command)[0]

            
                if not 'MC' in file:
                    name = file.replace(".5.4", "")
                    byte_command = bytes("%s\n\n\n\n\n"%name, 'utf-8')

                    native_program = "./COREX_Files/a.out_native"
                    denatured_program = "./COREX_Files/a.out_denatured"

                    ps = subprocess.Popen(native_program , shell=False, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
                    ps.communicate(input = byte_command)[0]

                    ps = subprocess.Popen(denatured_program , shell=False, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
                    ps.communicate(input = byte_command)[0]

            except:
                print("%s encountered an error and no ThermoDescript file was created" %check)
                problemos.append(check[0:4])

# Move all of the files generated into the COREX Output directory
for file in os.listdir(i_am_here):
    if file.endswith(".5.4") or file.endswith('.info') or file.endswith('DescriptN') or file.endswith('DescriptD'):
        shutil.move('%s/%s'%(i_am_here, file), '%s/%s'%(cor_dir, file))                
                
print(problemos)



In [ ]:
"""
FIND THE LOG ODDS SCORE FOR THE NATIVE STARTING PROTEIN STRUCTURES
"""

scores = []

for file in os.listdir(cor_dir):
    if file.endswith("0.500.ThermoDescriptN"):
        other_file = file.replace("0.500.ThermoDescriptN", "1.500.ThermoDescriptD")
        
        name = file[0:4]

        n_name = (os.path.join(cor_dir, file))
        d_name = (os.path.join(cor_dir, other_file))

        native_df = pd.read_csv(n_name, delim_whitespace=True)
        denat_df = pd.read_csv(d_name, delim_whitespace=True)

        native_df.drop(native_df[native_df['Res'] == "0"].index, inplace = True)
        denat_df.drop(denat_df[denat_df['Res'] == "0"].index, inplace = True)

        native_lo = native_df['Odds'].tolist()
        denat_lo = denat_df['Odds'].tolist()

        native_stability = native_df['ln(k_f)'].tolist()
        denat_stability = denat_df['ln(k_f)'].tolist()

        native_cluster = native_df['NTE'].tolist()
        denat_cluster = denat_df['DTE'].tolist()

        protein_len = len(native_lo)

        native_raw_score = sum(native_lo)
        denat_raw_score = sum(denat_lo)

        # Establish mu and sigma based off of equations shared by Jamie and published by Hoffmann 2016
        denat_mu = 0.105257632276167 - (0.173352418832684 * protein_len)
        denat_sigma = 0.609622745362049 * math.sqrt(protein_len)

        nat_mu = -0.0147176244982114 - (0.133774947797257 * protein_len)
        nat_sigma = 0.540008754110101 * math.sqrt(protein_len)


        n_cdf = scipy.stats.norm.cdf(native_raw_score, nat_mu, nat_sigma)
        d_cdf = scipy.stats.norm.cdf(denat_raw_score, denat_mu, denat_sigma)

        try:
            # Calculate -logP values 
            native_logp = -(math.log(1 - (n_cdf), 10))
            denat_logp = -(math.log(1 - (d_cdf), 10))

            total_logp = native_logp + denat_logp

            scores.append([name, native_logp, denat_logp, total_logp])
            
            df = pd.DataFrame(scores, columns = ['PDB', 'Native LogP', 'Denatured LogP', 'Total LogP'])
            df.to_csv('%s_corex_data.csv'%date)

        except:
            pass

       